In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import os

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

#### Data Loading and Merging

- Note: Change your file path

In [ ]:
riskfutures = pd.read_csv('/kaggle/input/forecasting-the-future-the-helios-corn-climate-challenge/corn_climate_risk_futures_daily_master.csv')
marketshare = pd.read_csv('/kaggle/input/forecasting-the-future-the-helios-corn-climate-challenge/corn_regional_market_share.csv')

In [ ]:
mergedf = riskfutures.copy()
mergedf['day_of_year'] = pd.to_datetime(mergedf['date_on'],format='%Y-%m-%d').dt.dayofyear
mergedf['quarter'] = pd.to_datetime(mergedf['date_on'],format='%Y-%m-%d').dt.quarter

In [ ]:
mergedf = mergedf.merge(marketshare[['region_id','percent_country_production']],how='left',on='region_id')

In [ ]:
mergedf['percent_country_production'] = mergedf['percent_country_production'].fillna(0.0)

#### Introduction of Climate Risk (Coldwave) by Tim

In [ ]:
# Total cnt locations for each rows
mergedf['total_location_by_region'] = mergedf['climate_risk_cnt_locations_heat_stress_risk_low'] + \
                                    mergedf['climate_risk_cnt_locations_heat_stress_risk_medium'] + \
                                    mergedf['climate_risk_cnt_locations_heat_stress_risk_high']

# Climate Risk for Coldwave, and Flood:
for i in range(1, 5):
    mergedf[f'medium_coldstress_lag_{i}'] = mergedf['climate_risk_cnt_locations_unseasonably_cold_risk_medium'].shift(i)
    mergedf[f'medium_coldstress_lag_{i}'] = mergedf[f'medium_coldstress_lag_{i}'].fillna(0)

for j in range(1, 3): 
    mergedf[f'high_coldstress_lag_{j}'] = mergedf['climate_risk_cnt_locations_unseasonably_cold_risk_high'].shift(j)
    mergedf[f'high_coldstress_lag_{j}'] = mergedf[f'high_coldstress_lag_{j}'].fillna(0)

mergedf['medium_coldstress_4days_average'] = mergedf[[f'medium_coldstress_lag_{i}' for i in range(1, 5)]].mean(axis=1)
mergedf['medium_coldstress_2days_average'] = mergedf[[f'medium_coldstress_lag_{i}' for i in range(1, 3)]].mean(axis=1)
mergedf['high_coldstress_2days_average'] = mergedf[[f'high_coldstress_lag_{i}' for i in range(1, 3)]].mean(axis=1)

mergedf['climate_risk_cnt_locations_coldwave_risk_high'] = (mergedf['medium_coldstress_4days_average'] + mergedf['high_coldstress_2days_average']) / 2
mergedf['climate_risk_cnt_locations_coldwave_risk_medium'] = (mergedf['medium_coldstress_2days_average'] + mergedf['high_coldstress_lag_1']) / 2


### 1 Baseline Feature Engineering

#### 1.1 Production-Weighted Risk Scores

In [ ]:
risk_categories = ['heat_stress', 'unseasonably_cold', 'excess_precip', 'drought', 'coldwave']
for risk in risk_categories:
    medium = f'climate_risk_cnt_locations_{risk}_risk_medium'
    high = f'climate_risk_cnt_locations_{risk}_risk_high'
    
    risk_scores = (1*mergedf[medium]+2*mergedf[high])/\
                           (mergedf['total_location_by_region'])
    ## define regional daily risk score as normalized weighted sum of number of locations
    
    production_weighted_risk_scores = (risk_scores*mergedf['percent_country_production'])/100
    ## use marketshare data to get production-weighted regional daily risk scores
    
    mergedf[f'climate_risk_{risk}_score'] = risk_scores
    mergedf[f'climate_risk_{risk}_weighted_score'] = production_weighted_risk_scores
    ## iterate for all five climate risk types; total 10 new engieered features

#### 1.2 Composite Risk Indices

In [ ]:
mergedf['climate_risk_temperature_stress'] = \
mergedf[[f'climate_risk_{risk}_score' for risk in risk_categories[:2]]].max(axis=1) 
## maximum of temperature-related risk scores
mergedf['climate_risk_precipitation_stress'] = \
mergedf[[f'climate_risk_{risk}_score' for risk in risk_categories[2:4]]].max(axis=1)
## maximum of precipitation-related risk scores
mergedf['climate_risk_overall_stress'] = \
mergedf[[f'climate_risk_{risk}_score' for risk in risk_categories]].max(axis=1)
## maximum of all risk scores
mergedf['climate_risk_avg_stress'] = \
mergedf[[f'climate_risk_{risk}_score' for risk in risk_categories]].mean(axis=1)
## average of all risk scores
## total 4 new engineered features

#### 1.3 Risk Temporal Summaries

In [ ]:
mergedf = mergedf.sort_values(['region_name','date_on'])
window_period = [7,14,30,60,90,120,240]
## three periods to compute risk scores moving avg and maximum 
for window in window_period:
    for risk in risk_categories:
        mergedf[f'climate_risk_{risk}_ma_{window}d'] = \
        mergedf.groupby(['region_name'])[f'climate_risk_{risk}_score']\
               .rolling(window=window,min_periods=1).mean().reset_index(level=0,drop=True)
## compute risk score moving avg with different windows for different risk types in each region

        mergedf[f'climate_risk_{risk}_max_{window}d'] = \
        mergedf.groupby(['region_name'])[f'climate_risk_{risk}_score']\
               .rolling(window=window,min_periods=1).max().reset_index(level=0,drop=True)
## compute maximum risk scores with different windows for different risk types in each region
## total 7*5*2 = 70 new features

#### 1.4 Risk Momentum

In [ ]:
features_change1d = mergedf.groupby('region_name')[[f'climate_risk_{risk}_score' for risk in risk_categories]]\
       .diff(periods=1)\
       .rename(columns=dict(zip([f'climate_risk_{risk}_score' for risk in risk_categories],\
                                [f'climate_risk_{risk}_change_1d' for risk in risk_categories])))
## Daily Change of risk scores for each risk type in each region 

features_acceleration = features_change1d.diff(periods=1)
## Acceleration of daily Change of risk scores for each risk type in each region

features_change1w = mergedf.groupby('region_name')[[f'climate_risk_{risk}_score' for risk in risk_categories]]\
       .diff(periods=7)\
       .rename(columns=dict(zip([f'climate_risk_{risk}_score' for risk in risk_categories],\
                                [f'climate_risk_{risk}_change_1d' for risk in risk_categories])))
## Weekly Change of risk scores for each risk type in each region 

mergedf = pd.concat([mergedf,\
           features_change1d,\
           features_change1w,\
           features_acceleration],axis=1)
## 15 new features in Risk Momentum category

#### 1.5 Cross-Regional features

In [ ]:
feature_country = pd.concat([\
mergedf.groupby(['country_name', 'date_on'])\
[[f'climate_risk_{risk}_score' for risk in risk_categories]]\
.agg(['mean','max','std']),
## compute country-wide daily avg, max, and std risk scores
mergedf.groupby(['country_name', 'date_on'])\
[[f'climate_risk_{risk}_weighted_score' for risk in risk_categories]]\
.agg('sum')],axis=1)
## compute country-wide daily production-weighted sum risk scores
feature_country.columns = [f'climate_risk_{risk}_score_country_{metric}'\
                          for risk in risk_categories \
                          for metric in ['mean','max','std']]+\
                          [f'climate_risk_{risk}_weighted_score_country_sum'\
                          for risk in risk_categories]
## rename new features
mergedf = mergedf.merge(feature_country.reset_index(),\
              how='left',\
              on=['country_name','date_on'])
## add 4*5=20 new features

#### Columns to Drop (Just select the drought, excess precipitation, and coldwave risks)

In [ ]:
cols_to_drop = ['climate_risk_cnt_locations_heat_stress_risk_low',
 'climate_risk_cnt_locations_heat_stress_risk_medium',
 'climate_risk_cnt_locations_heat_stress_risk_high',
 'climate_risk_cnt_locations_unseasonably_cold_risk_low',
 'climate_risk_cnt_locations_unseasonably_cold_risk_medium',
 'climate_risk_cnt_locations_unseasonably_cold_risk_high',
 'climate_risk_cnt_locations_excess_precip_risk_low',
 'climate_risk_cnt_locations_excess_precip_risk_medium',
 'climate_risk_cnt_locations_excess_precip_risk_high',
 'climate_risk_cnt_locations_drought_risk_low',
 'climate_risk_cnt_locations_drought_risk_medium',
 'climate_risk_cnt_locations_drought_risk_high',
 'climate_risk_cnt_locations_coldwave_risk_high',
 'climate_risk_cnt_locations_coldwave_risk_medium']

#### Non-Linear Transformation by William and Tim

In [ ]:
duplicates = mergedf.columns[mergedf.columns.duplicated()].tolist()
print(duplicates)

In [ ]:
import collections

counts = collections.defaultdict(int)
new_cols = []

for col in mergedf.columns:
    counts[col] += 1
    if counts[col] == 1:
        new_cols.append(col)       # keep first occurrence as-is
    else:
        new_cols.append(f"{col}__{counts[col]}")  # suffix duplicates

mergedf.columns = new_cols

In [ ]:
mergedf_1 = mergedf.copy()
mergedf_1 = mergedf.dropna()
print(mergedf_1.shape)

In [ ]:
pldf = pl.from_pandas(mergedf)
del mergedf

In [ ]:
pldf.shape

In [ ]:
climate_risk_cols = [c for c in pldf.columns if c.startswith('climate_risk_')]
climate_risk_selected_cols = [item for item in climate_risk_cols if item not in cols_to_drop]

In [ ]:
exprs = []

for feature_name in climate_risk_selected_cols:
    col = pl.col(feature_name)
    
    # log1p of non-negative values, fill nulls
    exprs.append(
        pl.when(col >= 0)
          .then((col + 1).log()) 
          .otherwise(0)
          .alias(f"{feature_name}_log1p")
    )

    # signed sqrt
    exprs.append(
        (col.sign() * col.abs().sqrt()).fill_null(0).alias(f"{feature_name}_ssqrt")
    )

    # threshold magnitude (>1)
    exprs.append(
        pl.when(col > 1)
          .then(col)
          .otherwise(0)
          .alias(f"{feature_name}_thresh_mag")
    )

    # tangent
    exprs.append(
        pl.col(feature_name).tan().fill_null(0).alias(f"{feature_name}_tangent")
    )

    # sine
    exprs.append(
        pl.col(feature_name).sin().fill_null(0).alias(f"{feature_name}_sin")
    )

    # cosine
    exprs.append(
        pl.col(feature_name).cos().fill_null(0).alias(f"{feature_name}_cos")
    )

# Add all new columns at once
pldf = pldf.with_columns(exprs)

In [ ]:
pldf.shape

In [ ]:
climate_risk_cols = [c for c in pldf.columns if c.startswith('climate_risk_')]
climate_risk_selected_cols = [item for item in climate_risk_cols if item not in cols_to_drop]

In [ ]:
std_multiplier = [1, 2, 3]
exprs = []

for multiplier in std_multiplier:
    for col_name in climate_risk_selected_cols:
        col = pl.col(col_name)
        # compute threshold: keep values > multiplier * std, else 0
        exprs.append(
            pl.when(col > multiplier * col.std())
              .then(col)
              .otherwise(0)
              .alias(f"{col_name}_above_{multiplier}_std")
        )

# add all new columns at once
pldf = pldf.with_columns(exprs)

In [ ]:
pldf.shape

In [ ]:
climate_risk_cols = [c for c in pldf.columns if c.startswith('climate_risk_')]
climate_risk_selected_cols = [item for item in climate_risk_cols if item not in cols_to_drop]

In [ ]:
pldf = pldf.sort(["region_name", "date_on"])

In [ ]:
window_period = [7, 14, 30]

exprs = []

for window in window_period:
    for col_name in climate_risk_selected_cols:
        exprs.append(
            pl.col(col_name)
              .shift(window)
              .alias(f"{col_name}_lag_{window}d")
        )

# Add lag features to mergedf
pldf = pldf.with_columns(exprs)

In [ ]:
print(pldf.shape)

In [ ]:
testing = [c for c in pldf.columns if c.startswith('futures_')]
print(testing)

### Run the overall cfcs score for all features ~13000

In [ ]:
def compute_partial_correlations(df,by=['crop_name','country_name','date_on_month']):
    ## df: must be a dataframe with non-zero rows and at least two columns 
    ## each with name starting with climate_risk and futures
    
    ## by: groupby keyword value to partition df to compute partial
    ## correlations of each group
    ## default to groupings shown in Helios sample notebook

    def _climate_futures_corr_table(df,climate_risk_columns,futures_columns):
        ## compute correlations of each climate-futures variable pair
        ## Note: dataframe must contain at least two non-null pairs
        ## to produce a non-null correlation
        
        corr_matrix = df[climate_risk_columns+futures_columns]\
                      .corr(method='pearson',min_periods=2,numeric_only=True)
        ## corr() auto drop nan values before computing
        ## number pairs must be at least two for computation
        ## according to formula
        corr_table = corr_matrix.loc[climate_risk_columns,futures_columns]\
        .rename_axis(index='climate_variable',columns='futures_variable')\
        .stack().reset_index(name='correlation')
        ## drop nan correlations in stack operation
        return corr_table.round(5)

    climate_risk_columns = [c for c in df.columns if c.startswith('climate_risk')]
    futures_columns = [c for c in df.columns if c.startswith('futures')]
    df_default = pd.DataFrame([],columns=['correlation'])
    
    if len(climate_risk_columns)==0:
        print('input dataframe must have at least one column with name starting with climate_risk')
        return df_default
    
    if len(futures_columns)==0:
        print('input dataframe must have at least one column with name starting with futures')
        return df_default
    
    if by==None:
        corr_tables = _climate_futures_corr_table(df,climate_risk_columns,futures_columns)
    else:
        try:
            corr_tables = df.groupby(by=by).apply(_climate_futures_corr_table,\
                                                  climate_risk_columns,\
                                                  futures_columns,\
                                                  include_groups=False
                                                 )
            corr_tables = corr_tables\
                          .reset_index(level=len(corr_tables.index.levels)-1,drop=True)\
                          .reset_index()
            ## compute and combine the correlation table for each group
        except KeyError:
            print('illegal by values')
            return df_default

    return corr_tables

In [ ]:
def cfcs(df):
    """
    Calculate the Climate-Futures Correlation Score (CFCS) for leaderboard ranking.
    
    CFCS = (0.5 × Avg_Sig_Corr_Score) + (0.3 × Max_Corr_Score) + (0.2 × Sig_Count_Score)

    Input dataframe must have correlation column for computation
    """

    # Remove null correlations
    valid_corrs = df["correlation"].dropna()
    
    if len(valid_corrs) == 0:
        return {'cfcs_score': 0.0, 'error': 'No valid correlations'}
    
    # Calculate base metrics
    abs_corrs = valid_corrs.abs()
    max_abs_corr = abs_corrs.max()
    significant_corrs = abs_corrs[abs_corrs >= 0.5]
    significant_count = len(significant_corrs)
    total_count = len(valid_corrs)
    
    # Calculate component scores - ONLY average significant correlations
    if significant_count > 0:
        avg_sig_corr = significant_corrs.mean()
        avg_sig_score = min(100, avg_sig_corr * 100)  # Cap at 100 when avg sig reaches 1.0
    else:
        avg_sig_corr = 0.0
        avg_sig_score = 0.0
    
    max_corr_score = min(100, max_abs_corr * 100)  # Cap at 100 when max reaches 1.0
    sig_count_score = (significant_count / total_count) * 100  # Percentage
    
    # Composite score: Focus more on quality of significant correlations
    cfcs = (0.5 * avg_sig_score) + (0.3 * max_corr_score) + (0.2 * sig_count_score)
    scoreboard = {'cfcs_score': cfcs, 'avg_sig_score': avg_sig_score,\
                  'max_corr_score': max_corr_score,\
                  'sig_count_score': sig_count_score
                 }
    print(f'{round(sig_count_score,2)}% of all correlations are significant')
    print(f'Average significant correlation is {round(avg_sig_corr,3)}')
    print(f'highest absolute correlation found is {round(max_abs_corr,3)}')
    print(f'final CFCS score is {round(cfcs,2)}')
    return scoreboard

In [ ]:
def sigcorr_report(df, features='climate_variable', sig_level=0.5):
    '''
        Generate a CFCS sub scores report for each feature
        with significant correlations.
        
        Input dataframe must contain a correlation column and feature columns.
    
        Returns a DataFrame with columns:
        ['climate_variable', 'avg_sig_corr','max_sig_corr','sig_corr_count','sig_corr_ratio(%)']
    '''
    
    # Absolute correlation
    df['correlation_abs'] = df.correlation.abs()
    
    try:
        # Mask correlations below threshold
        df.loc[df['correlation_abs'] < sig_level, ['correlation_abs']] = np.nan
        
        # Group by feature
        reportdf = df.groupby(features).agg({
            'correlation_abs': ['mean','max','count'],
            'correlation': 'count'
        })
        
    except TypeError:
        print('sig_level must be a number between 0 and 1')
        return None
    except KeyError:
        print('illegal features values')
        return None
    
    # Flatten multi-level columns
    reportdf.columns = ['avg_sig_corr','max_sig_corr','sig_corr_count','total_corr_count']
    
    # Compute ratio
    reportdf['sig_corr_ratio(%)'] = 100 * reportdf['sig_corr_count'] / reportdf['total_corr_count']
    
    # Keep only rows with avg_sig_corr not null
    reportdf = reportdf[reportdf['avg_sig_corr'].notnull()]\
                       .loc[:, ['avg_sig_corr','max_sig_corr','sig_corr_count','sig_corr_ratio(%)']]\
                       .round(3)
    
    # Add the feature name as a column
    reportdf[features] = reportdf.index
    reportdf = reportdf.reset_index(drop=True)  # optional: reset index
    
    # Reorder columns so feature name is first
    reportdf = reportdf[[features, 'avg_sig_corr','max_sig_corr','sig_corr_count','sig_corr_ratio(%)']]
    
    return reportdf

In [ ]:
pldf.shape

In [ ]:
climate_risk_cols = [c for c in pldf.columns if c.startswith("climate_risk")]
futures_col = [c for c in pldf.columns if c.startswith("futures_")]

### Reducing the computation time, therefore we did backtesting 200 by 200, instead of a 13000 overall features backtest

In [ ]:
chunk_size = 200  # number of climate_risk columns per chunk
column_chunks = [climate_risk_cols[i:i + chunk_size] for i in range(0, len(climate_risk_cols), chunk_size)]

#### Require to save the files into a folder, please change your file_path accordingly

In [ ]:
extra_cols = ['ID', 'crop_name', 'country_name', 'date_on_month']

for i, feature_chunk in enumerate(column_chunks):
    cols_for_corr = futures_col + feature_chunk + extra_cols
    pdf_chunk = pldf.select(cols_for_corr).to_pandas()

    corrtable = compute_partial_correlations(pdf_chunk)
    table = sigcorr_report(corrtable).sort_values(by='avg_sig_corr', ascending=False)

    # Create folder if it doesn't exist
    output_dir = "/kaggle/working/corr_files"
    os.makedirs(output_dir, exist_ok=True)  # exist_ok=True avoids error if folder already exists
    
    # Then you can save your CSV
    table.to_csv(f"{output_dir}/corr_report_cols_{i}.csv", index=False)

#### After the 1 hour running time of corr_report_cols_{i}, we merge them to have all features cfcs result.

In [ ]:
import glob, os

In [ ]:
import glob, os

# Folder containing your CSVs
csv_folder = "/kaggle/working/corr_files"

# Get all CSV file paths
csv_files = glob.glob(os.path.join(csv_folder, "*.csv"))

# Read all CSVs into a list of DataFrames
dfs = [pd.read_csv(f) for f in csv_files]

# Drop fully empty columns
dfs_clean = [df.dropna(axis=1, how='all') for df in dfs]

# Concatenate safely row-wise
result_table = pd.concat(dfs_clean, axis=0, ignore_index=True)

# Check
print(result_table.shape)
print(result_table.head())

result_table.to_csv('final_result_corr_2.csv', index=False)